In [197]:
import sys
sys.path.append('../../')

from IPython.core.display import display
import numpy as np
import pandas as pd
import networkx as nx
from sortedcontainers import SortedDict
import matplotlib.pyplot as plt
import seaborn as sns
from  matplotlib.ticker import PercentFormatter
import utils.graph_utils as graph_utils
from utils.IO_utils import load_ground_true
from sklearn import metrics

In [198]:
# settings
name = "Dblp"
come_model_type = "GMM"
ks = [2] # [2, 5, 10, 20]

In [199]:
# import graph and true labels

# graph
G = graph_utils.load_matfile(f"../../data/{name}/{name}.mat", undirected=True)

# labels_true
labels_true, _ = load_ground_true(path=f"../../data/{name}", file_name=name)
labels_true = np.array(labels_true) - 1  # 1..5 -> 0..4

In [200]:
# import node embeddings and predicted labels
dfs = {}
for k in ks:
    # import node embeddings
    df = pd.read_csv(
        f"./data/{name}_alpha-0.1_beta-0.1_ws-10_neg-5_lr-0.025_icom-219_ind-219_k-{k}_ds-0.0_type-{come_model_type}.txt",
        sep="\t| ",
        header=None
    )
    df = df.rename(columns={0: 'node'})
    df.set_index(['node'], inplace=True)

    # import predicted labels
    labels_pred = pd.read_csv(f"./data/labels_pred_{come_model_type}_{k}.txt", header=None)
    labels_pred = labels_pred.rename(columns={0: 'label_pred'})
    labels_pred.label_pred = labels_pred.astype(int)

    # join labels pred and true to embeddings
    df = df.join(labels_pred)
    df['label_true'] = labels_true

    print(f"k = {k}")
    display(df)
    display(df.describe())
    dfs[k] = df

/Users/anton/miniconda3/envs/MT/lib/python3.6/site-packages/ipykernel_launcher.py:8: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


k = 2


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
node,,,,,,,,,,,,,,,,,,,,,
0,0.305006,-0.466880,-0.267545,0.769969,0.285500,0.145671,0.145668,0.589570,0.057389,0.295072,...,0.022924,-0.109912,-0.527391,-0.116635,-0.446284,0.117288,-0.009859,-0.156843,1,0
1,0.274115,-0.487077,-0.116241,0.739823,0.195494,0.060871,0.084048,0.557284,-0.170716,0.435309,...,-0.059694,-0.219559,-0.513448,-0.049340,-0.475041,0.052064,0.117735,-0.032222,1,2
2,0.417771,-0.439148,-0.307630,0.787992,0.405380,0.128854,0.119469,0.588898,0.191887,0.283263,...,0.006722,-0.004066,-0.469753,-0.177917,-0.386277,0.056969,0.023117,-0.182263,1,2
3,0.306671,-0.452776,-0.340008,0.667185,0.238152,0.025103,0.174575,0.546116,0.084083,0.320654,...,-0.080180,-0.132030,-0.433514,-0.166257,-0.302751,0.087579,0.046053,-0.229579,1,0
4,0.218821,-0.380524,-0.374861,0.763850,0.261986,0.098802,0.026748,0.559295,0.217035,0.293235,...,0.038992,-0.054769,-0.509383,-0.214571,-0.346377,0.035100,-0.079765,-0.218875,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13179,0.233857,-0.375045,-0.195243,0.585578,0.377580,0.057074,0.045089,0.492758,0.113148,0.298501,...,-0.154959,-0.091873,-0.352352,-0.144252,-0.037096,0.055319,0.095775,-0.173232,1,0
13180,-0.128917,-0.556151,-0.205546,-0.030483,0.330267,0.210279,0.195406,0.034497,0.400683,0.501986,...,-0.314047,0.298250,-0.008569,0.241477,-0.022174,-0.508296,0.082054,0.339728,0,3
13181,-0.181398,-0.531019,-0.020660,0.022836,0.288119,0.118601,0.141081,0.019796,0.511212,0.564311,...,-0.129990,0.343729,-0.115812,0.100923,-0.177653,-0.494129,0.069173,0.356277,0,3


,1,2,3,4,5,6,7,8,9,10,...,121,122,123,124,125,126,127,128,label_pred,label_true
count,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,...,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000,13184.000000
mean,-0.051781,-0.302547,0.048093,0.296949,0.298413,-0.073264,0.185268,0.308804,0.186108,0.388025,...,-0.167927,0.258325,-0.256381,0.007310,-0.071415,-0.042185,0.105839,0.172020,0.390246,2.190382
std,0.167067,0.202704,0.173806,0.237132,0.161112,0.184697,0.166071,0.181970,0.202842,0.152158,...,0.165648,0.178690,0.209837,0.135285,0.182414,0.155744,0.183775,0.225410,0.487824,1.315339
min,-0.738680,-0.984638,-0.549125,-0.502673,-0.317483,-0.719233,-0.405115,-0.421883,-0.457067,-0.285070,...,-0.832682,-0.348916,-0.878605,-0.662500,-0.708759,-0.751996,-0.541037,-0.464598,0.000000,0.000000
25%,-0.166453,-0.457009,-0.067273,0.117994,0.195739,-0.211840,0.073862,0.186877,0.040458,0.289796,...,-0.280665,0.133314,-0.419754,-0.076076,-0.197066,-0.142536,-0.017640,-0.002352,0.000000,1.000000
50%,-0.048913,-0.287717,0.043891,0.295192,0.302898,-0.076033,0.177041,0.308882,0.173247,0.393894,...,-0.158188,0.253377,-0.263822,0.009260,-0.063929,-0.041156,0.088836,0.181500,0.000000,3.000000
75%,0.064279,-0.150278,0.160756,0.484957,0.405333,0.067229,0.292778,0.428846,0.313818,0.489991,...,-0.049476,0.369090,-0.088316,0.090387,0.059805,0.058771,0.213611,0.341792,1.000000,3.000000
max,0.573231,0.312830,0.729769,0.985580,0.866095,0.530542,0.880256,0.977802,0.926466,0.978306,...,0.388878,1.005381,0.456588,0.543176,0.444104,0.530632,0.853895,0.845029,1.000000,4.000000


# Community Detection

In [201]:
# NMI

for k in dfs:
    print("K =", k)
    df = dfs[k]
    labels_pred = df.label_pred.to_numpy()
    labels_true = df.label_true.to_numpy()

    nmi = metrics.normalized_mutual_info_score(labels_true, labels_pred)
    print("NMI: ", nmi)

K = 2
NMI:  0.42676536769046264


In [202]:
# Conductance

for k in dfs:
    print("K =", k)
    df = dfs[k]
    conductance_min = 1
    for i in range(k):
        nodes = df[df.label_pred == i].index.to_numpy()
        conductance_i = nx.conductance(G, nodes)
        print(f"  i{i} conductance: {conductance_i}")
        conductance_min = min(conductance_min, conductance_i)
    print(f"=>conductance: {conductance_min}")

K = 2
  i0 conductance: 0.07363816269138632
  i1 conductance: 0.014079610637928038
  i2 conductance: 0.41867257528452007
  i3 conductance: 0.08748040313549832
  i4 conductance: 0.04312428444218293
=>conductance: 0.014079610637928038


# Node Classification

In [203]:
# train LibSVM classifier on 90% of data and test on 10%.

for k in ks:
    df = dfs[k]

    train = df.

#TODO

SyntaxError: invalid syntax (<ipython-input-203-467d7ac4c663>, line 6)

In [ ]:
def emb_from_row(row):
    """get node embedding np array from df row"""
    return row.drop('label').to_numpy().flatten()
